In [173]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-dark')

import math
import openmc
import openmc.mgxs as mgxs
import openmc.data
import random
import h5py
import copy

In [241]:
class Sphere:
    def __init__(self, x_0, y_0, z_0, r):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 =z_0
        self.r = r
        
    def distance(self, particle):
    
       
        
        return distance
    
    
    def get_reflected_direction(self, angle):
        
        eps = random.uniform(0, math.pi * 2)
        tetta = random.uniform(0, math.pi)
        r_reflected = Direction(eps, tetta)
        
        return r_reflected  
    
    def get_sign(self, particle):
            
        sign = ((self.x_0 - particle.coordinates.x) * (self.x_0 - particle.coordinates.x) + 
        (self.y_0 - particle.coordinates.y) * (self.y_0 - particle.coordinates.y)  + 
                (self.z_0 - particle.coordinates.z) * (self.z_0 - particle.coordinates.z) - self.r * self.r)
        
        if sign == 0:
            return 0
            
        if sign < 0:
            return -1
        else:
            return 1  
        
    def get_xml(self):

        obj_xml = " Sphere with x_0 "+ str(x_0) + " y_0 "+ str(y_0) + " z_0 " + str(z_0) + " r = "+ str(r)
        
        return obj_xml
    

In [242]:
class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_s, number_of_production_neutrons, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = sig_s      
        self.number_of_production_neutrons = number_of_production_neutrons
        self.sig_t = sig_t
        self.get_virtual_cs()
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs for cross_section in self.sig_t]
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]

In [243]:
class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

In [244]:
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)

In [245]:
class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0. 
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self):  
        res = next(x for x, val in enumerate(self.energy_groups)
                                  if val > self.energy)        
        return res - 1
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        

In [246]:
class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, x_size, y_size, z_size):
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
         for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
             #   print("   " + str(i))
                if boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = self.surfaces[i].distance(particle)
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):

        if particle.coordinates.x > (self.x_size/2. + self.x_0):
            particle.coordinates.x -= self.x_size/2.
        
        if particle.coordinates.y > (self.y_size/2. + self.y_0):
            particle.coordinates.y -= self.y_size/2.
        
        if particle.coordinates.z > (self.z_size/2. + self.z_0):
            particle.coordinates.z -= self.z_size/2.
            
        
        if particle.coordinates.x < (-self.x_size/2. + self.x_0):
            particle.coordinates.x += self.x_size/2.
        
        if particle.coordinates.y < (-self.y_size/2. + self.y_0):
            particle.coordinates.y += self.y_size/2.
        
        if particle.coordinates.z < (-self.z_size/2. + self.z_0):
            particle.coordinates.z += self.z_size/2.
              

In [247]:
def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))

In [248]:
import random

def make_initial_sources(number_of_paricles, box_size, energy=10.0e6):
    
    step_x = 2 * box_size[0]/number_of_paricles
    step_y = 2 * box_size[1]/number_of_paricles
    step_z = 2 * box_size[2]/number_of_paricles
    
    x_coord = -box_size[0]
    y_coord = -box_size[1]
    z_coord = -box_size[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources

In [249]:
def get_free_path(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    sig_t = c_s.sig_t[energy_group_idx]
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  

In [250]:
def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)

In [251]:
def is_collision_virual(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    virtual_cs = c_s.virtual[energy_group_idx]
    total_cs = c_s.sig_t[energy_group_idx]
    if virtual_cs / total_cs >= random_number:
        return True
    else:
        return False

In [252]:
def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)

In [253]:
def process_real_collision(particle, free_path, universe, c_s):
    
    energy_group_idx = particle.get_energy_group()
    weight_before_collision = particle.get_weight()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        set_random_direction(particle)
        
    if type_collision == 'fission':    
        particle.set_particle_fission(number_of_production_neutrons)
        particle.set_terminated()
        
    return weight_before_collision
        

In [254]:
def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles

In [255]:
def process_one_particle_history_black_boundary(particle, universe, c_s):
    
    sum_collisions = 0.
    while not particle.is_terminated():
        
        free_path = get_free_path(particle, c_s)
        set_random_direction(particle)
        move_particle(particle, free_path)
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                
            
        if is_collision_virual(particle, c_s):
            process_virtual_collision(particle, free_path)
        else:
            sum_collisions += process_real_collision(particle, free_path, universe, c_s)
        
            
    return particle, sum_collisions

In [256]:
def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights

In [257]:
def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False

In [258]:
def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))

In [259]:
def splitting_without_random(particles):
     
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)          
            particle.set_weight(current_weight / n_value)
            for j in range(0, n_value - 2):
                particles.append(copy.deepcopy(particle))
            additonal_particle = copy.deepcopy(particle)
            additonal_particle.set_weight(current_weight - n_value)
            particles.append(additonal_particle)

In [260]:
def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles

In [261]:
def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])

In [262]:
def flux_estimator(cell_volume, start_weight, particles_current_iteration, cross_section):
    
    sum_weighted_paths = 0.
    
    for i in range(0, len(particles_current_iteration)):
        particle = particles_current_iteration[i]
        energy_group_idx = particle.get_energy_group() 
  
        number_of_production_neutrons = cross_section.number_of_production_neutrons[energy_group_idx]   
        sum_weighted_paths += particles_current_iteration[i].get_weight()* (
            particles_current_iteration[i].get_path())
        
    total_flux = sum_weighted_paths/(cell_volume * start_weight)
    
    return total_flux

In [263]:
def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")

In [264]:
def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_current = 0.
        
        for i in range(0, idx - number_inactive):
            std_current += (k_effective[i] - k_effective_exp[idx - number_inactive - 1])**2
            
        std_k_effective.append(math.sqrt(std_current)/(idx * max(idx - 1, 1)))

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))
            

In [265]:
def calculate_flux(weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]
    flux = sum_collisions / ( sum(weight_previos) * volume * sig_t)

In [266]:
import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    initial_sources = make_initial_sources(number_of_particles, energy=10.0e6, box_size=[0.5, 0.5, 0.5])
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []
        total_collisions = 0.

        for j in range(0, batch_size):
            
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history_black_boundary(particle, universe, c_s)
            batch_particles.append(terminate_particle)
            total_collisions += sum_collisions
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        

        batch_particles = normalise_weights(batch_particles, initial_size) 
        

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std

In [267]:
def save_simulation_results_to_file(number_of_particles, number_interations, number_inactive,
                                   universe, k_effective, k_effective_exp, k_effective_std, file_name):
    
    f = open(file_name, "w")
    f.write("number of particles  " + str(number_of_particles) + "\n")
    f.write("number of interations  " + str(number_interations) + "\n")
    f.write("number inactive  " + str(number_inactive) + "\n")
    universe_discription = universe.get_xml()
    f.write("geometry \n")
    for xml_cell in universe_discription:
        f.write(xml_cell + "\n")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + "  average k effective "+ str(k_effective_exp[i]) + "  std "+ 
                str(k_effective_std[i]) + '\n')
            
    
    f.close()

In [270]:
test_number_of_particles = 20
test_number_interations = 1000
test_number_inactive = 10

In [271]:
boundaries_type = ["black"]

cylinder = Cell(surfaces, signs)

cylinder.set_boundaries_type(boundaries_type)

cylinder.set_box_sizes(0., 0., 10.)
cylinder.set_zero_point(0., 0., 0.)

In [272]:
test_sphere = Sphere(0, 0, 0, 6.082547)
boundaries_type = ["black"]

surfaces = [test_sphere]
signs = [-1]

sphere = Cell(surfaces, signs)
sphere.set_boundaries_type(boundaries_type)
k_effective, k_effective_exp, k_effective_std = simulation_black_boundaries(sphere,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs_1_4)

 num1  0
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
 keff_cycle , k_effective_exp, std_k_effective 0.9833530373780118   0.9833530373780118  0.0
i == 12
 keff_cycle , k_effective_exp, std_k_effective 0.9716014771638867   0.9774772572709493  6.295157512825709e-05
i == 13
 keff_cycle , k_effective_exp, std_k_effective 1.0145925987890603   0.9898490377769863  0.00020143054512202096
i == 14
 keff_cycle , k_effective_exp, std_k_effective 1.000336614611899   0.9924709319857145  0.00017972217990048565
i == 15
 keff_cycle , k_effective_exp, std_k_effective 0.9913396714197217   0.992244679872516  0.00015583372820686514
i == 16
 keff_cycle , k_effective_exp, std_k_effective 1.0078178547649792   0.9948402090212598  0.00014866502003044022
i == 17
 keff_cycle , k_effective_exp, std_k_effective 1.0023633826161178   0.9959149481062395  0.00013365105297316113
i == 18
 keff_cycle , k_effective_exp, std_k_effective 0.9965559967684461   0.9959950791890153  0.0001

 keff_cycle , k_effective_exp, std_k_effective 0.9793920493150257   1.0011523902517159  2.0914222668648304e-05
i == 81
 keff_cycle , k_effective_exp, std_k_effective 0.9741460713116638   1.0007720195624195  2.0813355037083387e-05
i == 82
 keff_cycle , k_effective_exp, std_k_effective 1.0043664939637806   1.0008219428179939  2.0312822313951972e-05
i == 83
 keff_cycle , k_effective_exp, std_k_effective 0.982525392441374   1.0005713051416019  2.0002335209606606e-05
i == 84
 keff_cycle , k_effective_exp, std_k_effective 0.9797265781446778   1.0002896196416435  1.9750598236204612e-05
i == 85
 keff_cycle , k_effective_exp, std_k_effective 0.9912448234965715   1.0001690223597093  1.9326883647378882e-05
i == 86
 keff_cycle , k_effective_exp, std_k_effective 0.9937282904129422   1.0000842758867254  1.8897701682000514e-05
i == 87
 keff_cycle , k_effective_exp, std_k_effective 0.9946071424834647   1.0000131442840856  1.8477589913811186e-05
i == 88
 keff_cycle , k_effective_exp, std_k_effective 0.

 keff_cycle , k_effective_exp, std_k_effective 1.0154166260963955   1.000132451594442  8.334527591716538e-06
i == 150
 keff_cycle , k_effective_exp, std_k_effective 0.9787351766975136   0.999979613916607  8.278546485611285e-06
i == 151
 keff_cycle , k_effective_exp, std_k_effective 1.014662639190061   1.0000837488476242  8.194396435116396e-06
i == 152
 keff_cycle , k_effective_exp, std_k_effective 1.001528796807667   1.000093925241709  8.086818791549183e-06
i == 153
 keff_cycle , k_effective_exp, std_k_effective 1.0029093676606018   1.0001136136502329  7.982020453711511e-06
i == 154
 keff_cycle , k_effective_exp, std_k_effective 0.9931303525217579   1.000065118781285  7.883891958032684e-06
i == 155
 keff_cycle , k_effective_exp, std_k_effective 1.0057423371769825   1.0001042720116  7.785772845412955e-06
i == 156
 keff_cycle , k_effective_exp, std_k_effective 1.0087433690523833   1.0001634439091396  7.694198117516853e-06
i == 157
 keff_cycle , k_effective_exp, std_k_effective 0.99377259

 keff_cycle , k_effective_exp, std_k_effective 1.0110786307827941   1.0006616122536407  4.706262303193895e-06
i == 220
 keff_cycle , k_effective_exp, std_k_effective 1.0023473643270961   1.0006696396444668  4.663608727988759e-06
i == 221
 keff_cycle , k_effective_exp, std_k_effective 0.9927703277524741   1.0006322021473484  4.624245626445101e-06
i == 222
 keff_cycle , k_effective_exp, std_k_effective 1.0145035557064854   1.0006976330603632  4.591258218450108e-06
i == 223
 keff_cycle , k_effective_exp, std_k_effective 0.9867364007067119   1.000632087368562  4.558771108031507e-06
i == 224
 keff_cycle , k_effective_exp, std_k_effective 1.000743646113337   1.0006326086711077  4.518068343961736e-06
i == 225
 keff_cycle , k_effective_exp, std_k_effective 0.9617366982629202   1.0004516974599067  4.543619340100368e-06
i == 226
 keff_cycle , k_effective_exp, std_k_effective 0.9947652207748955   1.0004253711789577  4.5047921384605264e-06
i == 227
 keff_cycle , k_effective_exp, std_k_effective 0.

 keff_cycle , k_effective_exp, std_k_effective 0.9684965931684776   1.0008390621349905  3.1505707631735468e-06
i == 289
 keff_cycle , k_effective_exp, std_k_effective 1.0169571790668377   1.00089683316342  3.134733342461945e-06
i == 290
 keff_cycle , k_effective_exp, std_k_effective 0.9778721313495158   1.0008146020855133  3.1251697734051648e-06
i == 291
 keff_cycle , k_effective_exp, std_k_effective 0.989487407495276   1.0007742917844804  3.1065816485024925e-06
i == 292
 keff_cycle , k_effective_exp, std_k_effective 1.0256504671102236   1.0008625051721602  3.0991130662812436e-06
i == 293
 keff_cycle , k_effective_exp, std_k_effective 1.0136667513372275   1.000907749858256  3.0815821537502533e-06
i == 294
 keff_cycle , k_effective_exp, std_k_effective 1.0098944480318812   1.0009393931616841  3.062390234263685e-06
i == 295
 keff_cycle , k_effective_exp, std_k_effective 1.0038286196008936   1.000949530798313  3.041810047720764e-06
i == 296
 keff_cycle , k_effective_exp, std_k_effective 0

...

 keff_cycle , k_effective_exp, std_k_effective 1.0059331806961307   1.0004470003630141  8.063797350667197e-07
i == 704
 keff_cycle , k_effective_exp, std_k_effective 0.9935126049325363   1.0004370084387628  8.042107735343932e-07
i == 705
 keff_cycle , k_effective_exp, std_k_effective 1.017380679910665   1.0004613878221755  8.026546012229498e-07
i == 706
 keff_cycle , k_effective_exp, std_k_effective 1.0304540488572593   1.0005044807259615  8.026427162976423e-07
i == 707
 keff_cycle , k_effective_exp, std_k_effective 1.0160697794148523   1.0005268125748694  8.009785534967349e-07
i == 708
 keff_cycle , k_effective_exp, std_k_effective 1.0058039435884178   1.0005343729344878  7.987853768949708e-07
i == 709
 keff_cycle , k_effective_exp, std_k_effective 1.0119556651876656   1.0005507124083834  7.968565388664783e-07
i == 710
 keff_cycle , k_effective_exp, std_k_effective 1.0035550566235338   1.0005550043286908  7.946342534927139e-07
i == 711
 keff_cycle , k_effective_exp, std_k_effective 0.

 keff_cycle , k_effective_exp, std_k_effective 0.972395497756688   1.0004396596815084  7.071426695821156e-07
i == 774
 keff_cycle , k_effective_exp, std_k_effective 1.005070382711231   1.0004457208373063  7.053578368752971e-07
i == 775
 keff_cycle , k_effective_exp, std_k_effective 1.0181650287592137   1.000468883331322  7.041566172330562e-07
i == 776
 keff_cycle , k_effective_exp, std_k_effective 1.0099026515348344   1.0004811989556086  7.025167022715328e-07
i == 777
 keff_cycle , k_effective_exp, std_k_effective 0.9962161277729856   1.000475638236987  7.00744079250664e-07
i == 778
 keff_cycle , k_effective_exp, std_k_effective 1.0074444259182254   1.0004847121792804  6.990376199552512e-07
i == 779
 keff_cycle , k_effective_exp, std_k_effective 1.018257438468619   1.0005078236568998  6.978585212465534e-07
i == 780
 keff_cycle , k_effective_exp, std_k_effective 1.0024818942847977   1.0005103873849879  6.960767124494166e-07
i == 781
 keff_cycle , k_effective_exp, std_k_effective 1.01298

 keff_cycle , k_effective_exp, std_k_effective 1.0340025813675675   1.000300836067059  6.264624470634978e-07
i == 844
 keff_cycle , k_effective_exp, std_k_effective 1.0005070233999505   1.0003010832940769  6.249780060021532e-07
i == 845
 keff_cycle , k_effective_exp, std_k_effective 1.0282631814404195   1.000334570836767  6.247288300976365e-07
i == 846
 keff_cycle , k_effective_exp, std_k_effective 0.9951476774593068   1.0003283664188514  6.232941122955256e-07
i == 847
 keff_cycle , k_effective_exp, std_k_effective 0.9920114077214368   1.0003184297895833  6.219305287382694e-07
i == 848
 keff_cycle , k_effective_exp, std_k_effective 1.0131108738609385   1.0003336952359692  6.207189782087619e-07
i == 849
 keff_cycle , k_effective_exp, std_k_effective 1.0207389816446524   1.0003580161971235  6.199042320471075e-07
i == 850
 keff_cycle , k_effective_exp, std_k_effective 1.004721519840533   1.0003632108443181  6.184751567012957e-07
i == 851
 keff_cycle , k_effective_exp, std_k_effective 0.96

 keff_cycle , k_effective_exp, std_k_effective 1.0123511624933117   1.0002720515419055  5.579964447483677e-07
i == 914
 keff_cycle , k_effective_exp, std_k_effective 1.0134232060144586   1.0002865992791539  5.569981951853015e-07
i == 915
 keff_cycle , k_effective_exp, std_k_effective 1.0128081830968252   1.0003004352833722  5.559821277157628e-07
i == 916
 keff_cycle , k_effective_exp, std_k_effective 1.0230522180923223   1.0003255476264283  5.554311984337596e-07
i == 917
 keff_cycle , k_effective_exp, std_k_effective 0.9904310593221133   1.0003146385985293  5.543448198447668e-07
i == 918
 keff_cycle , k_effective_exp, std_k_effective 1.0008633412077048   1.0003152428965572  5.531374805521478e-07
i == 919
 keff_cycle , k_effective_exp, std_k_effective 1.0108817217792105   1.000326867185757  5.520756355887584e-07
i == 920
 keff_cycle , k_effective_exp, std_k_effective 0.9948720187169796   1.0003208728467803  5.509132095118849e-07
i == 921
 keff_cycle , k_effective_exp, std_k_effective 1.

 keff_cycle , k_effective_exp, std_k_effective 1.0202180509776366   1.0003555677102174  4.958211882849193e-07
i == 984
 keff_cycle , k_effective_exp, std_k_effective 1.00379782541285   1.0003591018557028  4.948262056294849e-07
i == 985
 keff_cycle , k_effective_exp, std_k_effective 1.000307721854669   1.0003590491582657  4.938214852116888e-07
i == 986
 keff_cycle , k_effective_exp, std_k_effective 1.0168613258841697   1.0003759572286817  4.931123480986479e-07
i == 987
 keff_cycle , k_effective_exp, std_k_effective 1.0284576211218162   1.0004046999757574  4.929575281378365e-07
i == 988
 keff_cycle , k_effective_exp, std_k_effective 0.9779280922509892   1.00038171775927  4.92498741327709e-07
i == 989
 keff_cycle , k_effective_exp, std_k_effective 0.9983557980878841   1.0003796483826906  4.915071569014804e-07
i == 990
 keff_cycle , k_effective_exp, std_k_effective 1.0176667087097342   1.000397288240167  4.908315451650002e-07
i == 991
 keff_cycle , k_effective_exp, std_k_effective 1.003419